In [1]:
import json
import os
import numpy as np
import sys
import tensorflow as tf
from tqdm import tqdm_notebook

from src import model, sample, encoder

W0830 10:29:12.566389 13136 deprecation_wrapper.py:119] From C:\Users\AI-Machine\Documents\gpt-2\src\model.py:147: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.



In [6]:

def generateMoviePlots(
    movieSummaries,
    model_name='117M',
    seed=None,
    nsamples=1,
    batch_size=1,
    length=None,
    temperature=1,
    top_k=0,
    top_p=0.0
):
    """
    Interactively run the model
    :model_name=117M : String, which model to use
    :seed=None : Integer seed for random number generators, fix seed to reproduce
     results
    :nsamples=1 : Number of samples to return total
    :batch_size=1 : Number of batches (only affects speed/memory).  Must divide nsamples.
    :length=None : Number of tokens in generated text, if None (default), is
     determined by model hyperparameters
    :temperature=1 : Float value controlling randomness in boltzmann
     distribution. Lower temperature results in less random completions. As the
     temperature approaches zero, the model will become deterministic and
     repetitive. Higher temperature results in more random completions.
    :top_k=0 : Integer value controlling diversity. 1 means only 1 word is
     considered for each step (token), resulting in deterministic completions,
     while 40 means 40 words are considered at each step. 0 (default) is a
     special setting meaning no restrictions. 40 generally is a good value.
    :top_p=0.0 : Float value controlling diversity. Implements nucleus sampling,
     overriding top_k if set to a value > 0. A good setting is 0.9.
    """
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = encoder.get_encoder(model_name)
    hparams = model.default_hparams()
    with open(os.path.join('models', model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k, top_p=top_p
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join('models', model_name))
        saver.restore(sess, ckpt)

        with open("generated_movie_plots.txt", "w", encoding="utf-8") as f:
            # make a loop here over the movie-plots
            for i in tqdm_notebook(range(0, len(movieSummaries), 2)):
                # get the summary as input
                raw_text = "[WP] " + movieSummaries[i+1].strip()
                # write the name of the movie followed by the generated plot in a file
                f.write("[Movie] " + movieSummaries[i].strip() + "\n")
                f.write("[Plot] " + movieSummaries[i+1].strip() + "\n")
                print("======================================== INPUT ========================================\n", raw_text)
                while not raw_text:
                    print('Prompt should not be empty!')
                    raw_text = input("Model prompt >>> ")
                context_tokens = enc.encode(raw_text)
                generated = 0
                for _ in range(nsamples // batch_size):
                    out = sess.run(output, feed_dict={
                        context: [context_tokens for _ in range(batch_size)]
                    })[:, len(context_tokens):]
                    for i in range(batch_size):
                        generated += 1
                        text = enc.decode(out[i])
                        f.write(text+"\n")
                        print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                        print(text)
        print("=" * 80)

        
with open("data/movie_plot_data.txt", "r", encoding="utf-8") as f:
    movie_plots = f.readlines()
    
len(movie_plots)
generateMoviePlots(movie_plots, model_name="writingprompts774M", top_p=0.9, nsamples=1, length=None)

======================================== INPUT ========================================
 [WP] Chronicles the experiences of a formerly successful banker as a prisoner in the gloomy jailhouse of Shawshank after being found guilty of a crime he did not commit. The film portrays the man's unique way of dealing with his new, torturous life; along the way he befriends a number of fellow prisoners, most notably a wise long-term inmate named Red.
======================================== SAMPLE 1 ========================================
 
[Story] When you hear about the slam I was hit with in the final moments of my life, it usually comes as something that I did or wanted. It rarely happens. I don't even know how it happened. I was walking out the front door at the end of the day. Or, if you think about it, from the last corner of the building that still managed to maintain its old feel. I was coming back from my friend Dylan's party at the nearby town bar. My destination, I had decided, was a

 [WP] Set within a year after the events of Batman Begins, Batman, Lieutenant James Gordon, and new district attorney Harvey Dent successfully begin to round up the criminals that plague Gotham City until a mysterious and sadistic criminal mastermind known only as the Joker appears in Gotham, creating a new wave of chaos. Batman's struggle against the Joker becomes deeply personal, forcing him to "confront everything he believes" and improve his technology to stop him. A love triangle develops between Bruce Wayne, Dent and Rachel Dawes.
======================================== SAMPLE 1 ========================================
 Written with a heavy influence from the Batman books. 1/4 will be published in print, with an expanded story later that uses my own "voice" as the main character, and not the other way around. Thanks for reading! I hope you enjoy the story as much as I did making it.<|endoftext|>
[Prompt] You play a new console MMORPG, where you and your fellow players are just n

[Story] The sappy uplift of bells, triumphant cavalry charges, and spittle through the eyes of the last fighting Polish resistance fighters that escaped from the hated allied Germans. Shores of sound that brought another man's life to an end. Those were the least of my emotions when I had landed in Normandy. I woke up in front of my desk and rolled over to open a special version of the mass mail from my constituents. The sound of the assault rifle slapping against the door. Rifles. Mass mail, checking of my mail. My phone buzzed. A call from HR. Three dial-out numbers ringing. Making a fit of nervousness. What was the latest mailhead changes? After the last time the mail carrier was mended and properly fitted the employee's had a right to feel somewhat disappointed. Better be delivered, they said. Better be nicely delivered than the lost mail. The CEO turned off his light, facing the stairs. I sat up straighter in my bed and slowly sat up from the crumpled pillow lying over my bed. Per

 [WP] Blondie (The Good) is a professional gunslinger who is out trying to earn a few dollars. Angel Eyes (The Bad) is a hit man who always commits to a task and sees it through, as long as he is paid to do so. And Tuco (The Ugly) is a wanted outlaw trying to take care of his own hide. Tuco and Blondie share a partnership together making money off Tuco's bounty, but when Blondie unties the partnership, Tuco tries to hunt down Blondie. When Blondie and Tuco come across a horse carriage loaded with dead bodies, they soon learn from the only survivor (Bill Carson) that he and a few other men have buried a stash of gold in a cemetery. Unfortunately Carson dies and Tuco only finds out the name of the cemetery, while Blondie finds out the name on the grave. Now the two must keep each other alive in order to find the gold. Angel Eyes (who had been looking for Bill Carson) discovers that Tuco and Blondie met with Carson and knows they know the location of the gold. All he needs is for the two 

[Story] They call me the Bomb. They were supposed to rescue me from the scrap heap, but I was still pretty battered when I arrived. I was lost in my own mind, in a swamp of emotional turmoil. I haven't seen a human since my husband died. I'm afraid to tell my soulmate. He's gone into the god complex, and my wife just died of depression. I was scared to talk to a human. I just don't feel like. A few days after I came here, a human appeared, in a sweet yellow suit. He walked up to me and gave me a small box. I've never been able to use my legs and have seen them very rarely, but this was so much more than they are
======================================== INPUT ========================================
 [WP] Forrest Gump is a simple man with a low I.Q. but good intentions. He is running through childhood with his best and only friend Jenny. His 'mama' teaches him the ways of life and leaves him to choose his destiny. Forrest joins the army for service in Vietnam, finding new friends called

...

[Story] Satyajit from the great village knew the name of the "brave doctor". He knew that his father was Munna and that the name was given by Munna's parents. The name always had a special meaning. Since his mother's gift, he always went with his mother and often a dog of his own. For whenever his father called him to the shed, he would get his favourite into a bowl, if he had one of his own, and eat. After all, the dog didn't feed or care for the "brave doctor", though the thing did eat. From the day the dog got to him, Satyajit looked up at the big sky. He saw that it was just a blur. He understood the danger that might lay behind the sky. He knew that if he did not get out of the shed, he would have to look around for the mystery. He looked up. It was something fixed, something living in the sky above. A figure. A head. And a body. The body couldn't be a body, because it was based on a head. The figure moved out of the trees above the leaves of the trees and stretched its legs out. 

 [WP] Set in Hong Kong, 1962, Chow Mo-Wan is a newspaper editor who moves into a new building with his wife. At the same time, Su Li-zhen, a beautiful secretary and her executive husband also move in to the crowded building. With their spouses often away, Chow and Li-zhen spend most of their time together as friends. They have everything in common from noodle shops to martial arts. Soon, they are shocked to discover that their spouses are having an affair. Hurt and angry, they find comfort in their growing friendship even as they resolve not to be like their unfaithful mates.
======================================== SAMPLE 1 ========================================
 
[WP] A suicide note written by a young writer about his roommate for a client. 
[WP] A suicide note written by a young writer about his roommate for a client. 
[WP] A suicide note written by a young writer about his roommate for a client. 
[WP] A suicide note written by a young writer about his roommate for a client. 
[WP]

 [Story] "Alright girls, go get set for a few minutes and grab some lunch" "And on the weekends, remember this page and never forget to smile at that charming old man!" "And what's the meaning of that?!" The joke was thinly-veiled but Aibileen and Skeeter both knew what it meant. A couple days later, the people were well and truly set for breakfast. A pair of women worked as assistant chefs and the rest of the women spent the morning basking in the sun as Aibileen played bare-bones chess with her family as the rest of the boys waited by the boys' seats, and Skeeter let out a series of joyous exhales. Across the street, her sister gave a brilliant rendition of The Sound of Music while everyone else ate hotcakes out of plastic containers and prepared cookies with a mix of brown sugar and granulated sugar. "A cookie was better than anything else" she said, knocking the top cookie onto the table to make it fall apart into thousands of pieces. "Plus, we're using this place to make some boy 

 [WP] Based upon a real-life story that happened in the early seventies in which the Chase Manhattan Bank in Gravesend, Brooklyn, was held siege by a bank robber determined to steal enough money for his male lover to undergo a sex change operation. On a hot summer afternoon, the First Savings Bank of Brooklyn is held up by Sonny and Sal, two down-and-out characters. Although the bank manager and female tellers agree not to interfere with the robbery, Sonny finds that there's actually nothing much to steal, as most of the cash has been picked up for the day. Sonny then gets an unexpected phone call from Police Captain Moretti, who tells him the place is surrounded by the city's entire police force. Having few options under the circumstances, Sonny nervously bargains with Moretti, demanding safe escort to the airport and a plane out of the country in return for the bank employees' safety.
======================================== SAMPLE 1 ========================================
 The robb

[Story] I begin his work by knelling the coal and the coal takes a fortnight, with about six weeks passage. Each coal, the smallest of the morning I lose for many days, and so the valuable miner is long gone. Two weeks I lost it, then it was twice that by six months, and from my superstitious shack I knew it was over. I looked at Will, I looked at him and he looked at me, for the first time, and said, "Do you think I could make a perfectly lathe and blade for the sword? I have a hunk of iron that I wouldn't mind so much if I were you." "Why not?" I asked. "You're on a journey that took me five years of sailing to find Elizabeth Swann, and I've already died; I need the place to rest my head." I offered. "For five years?" "Five! I didn't think you'd travel the world like that," said Will. "But it was enough, I was to be enslaved. They are determined to see me dead; the moment the bells ring, they take me. I don't know what to do now." "What do you want?" "Elizabeth is everything to me," 